In [ ]:
# IMPORTS #

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [ ]:
# MODULE IMPORTS # (because Jupyter NoteBooks can't import other Notebooks classes)

# Should be -----------------------
# import Image3d
#----------------------------------

import SimpleITK
from scipy.signal import wiener

def rgb2grey(rgb):
    """
        Creates a luma transformation of an rgb array
        
        :param rgb: a list of arrays
        :type rgb: numpy.ndarray
        :return: the grey array
        :rtype: numpy.ndarray
        
        :Example:
        
        >>> import numpy
        >>> a = numpy.array([1,2,3])
        >>> print(rgb2grey(a))
        [1.8596 1.8596 1.8596]
    """
    luma = rgb[0] * luma_coeff[0] + rgb[1] * luma_coeff[1] + rgb[2] * luma_coeff[2]
    formula = [luma, luma, luma]
    return np.array(formula)

luma_coeff = [0.2126, 0.7152, 0.0722]

def get_AVGall(arr):
    return np.mean(np.concatenate((arr[:10], arr[-10:])))

class Image:
    
    def __init__(self, fn='', fromArray=np.array([])):
        assert fn or fromArray.ndim>=2, "Image must take a filename or a numpy array as a parameter"
        if fn:
            img = SimpleITK.ReadImage(fn)
            self.__filename = fn
            self.__width = img.GetWidth()
            self.__height = img.GetHeight()
            self.__array = SimpleITK.GetArrayFromImage(img)
            self.__array = self.__array / 65535.0
        else:
            self.__filename = None
            self.__width = fromArray.shape[1]
            self.__height = fromArray.shape[0]
            self.__array = fromArray / 65535.0
    
    def get_filename(self):
        # permet d'obtenir le nom du fichier image origine (s'il existe)
        return self.__filename
    
    def get_array(self):
        # getter pour l'array correspondant à l'image
        return self.__array
    
    def get_grey_array(self, arr):
        # permet d'obtenir la version grise d'un array
        grey_array = []
        for i in range(len(arr)): # not optimized
            inter = []
            for j in range(len(arr[i])):
                inter.append(rgb2grey(arr[i][j]))
            grey_array.append(np.array(inter))
        return np.array(grey_array)
    
    def wiener_filtered(self, arr):
        return wiener(arr)
    
    def __correct_column(self, col):
        # permet de corriger une colonne d'un array avec la méthode de streks correction
        arr = []
        start = np.mean(col[:10])
        end = np.mean(col[-10:])
        for i in range(10, len(col)-10):
            arr.append(col[i] * get_AVGall(self.get_array()) / (((len(col) - i) / len(col)) * start + (i / len(col)) * end))
        return np.concatenate((col[:10], np.array(arr), col[-10:]))

    def streaks_corrected(self, arr):
        # corrige un array en appliquant la streaks correction
        return np.apply_along_axis(self.__correct_column, 0, arr)
    
    def get_width(self):
        # getter pour la largeur de l'image
        return self.__width
    
    def get_height(self):
        # getter pour la hauteur de l'image
        return self.__height
    
    def show(self, mode=''):
        # affiche l'image avec le(s) modes désirés
        # 'r' pour afficher l'image à l'envers
        # 'w' pour que l'image soit passée à travers un filtre de Weiner
        # 'g' pour afficher l'image en gris (avec les coefficients luma)
        # 's' pour réaliser une streaks correction sur l'image
        
        array = self.get_array()
        dir = 'upper'
        
        if 'r' in mode:
            dir ='lower'
        
        if 'w' in mode:
            array = self.wiener_filtered(array)
        if 'g' in mode:
            array = self.get_grey_array(array)
        if 's' in mode:
            array = self.streaks_corrected(array)
        
        array[array > 1] = 1.0
        plt.imshow(array, origin=dir)

def get_med_img(arr):
    """
        Creates a median array from a list of arrays
        
        :param arr: a list of arrays
        :type arr: list or numpy.ndarray
        :return: the median array
        :rtype: numpy.ndarray
        
        :Example:
        
        >>> import numpy
        >>> a = numpy.array([[7,8,9],[16,17,18]])
        >>> b = numpy.array([[13,14,15],[4,5,6]])
        >>> c = numpy.array([[1,2,3],[10,11,12]])
        >>> print(get_med_img([a, b, c]))
        [[ 7.  8.  9.]
         [10. 11. 12.]]
    """
    l3d = np.array(arr)
    return np.median(l3d, axis=0)

class Image3d:
    
    def __init__(self, fns, path=''):
        arrays = []
        for i in range(len(fns)):
            img = SimpleITK.ReadImage(path + fns[i])
            arrays.append(SimpleITK.GetArrayFromImage(img))
        self.__arrays = arrays
    
    def get_2dImage(self):
        # retourne l'image 2D correspondant à l'image médiane
        return Image(fromArray=get_med_img(self.get_array()))
    
    def get_width(self):
        # getter pour la largeur de l'image
        return self.get_2dImage().get_width()
    
    def get_height(self):
        # getter pour la hauteur de l'image
        return self.get_2dImage().get_height()
    
    def get_array(self):
        # getter pour l'array correspondant à l'image 3D
        return self.__arrays

In [ ]:
# ACTUAL CLASS #

class Calibration:
    
    def __init__(self, img):
        self.__image = img
    
    def get_image(self):
        # getter for the calibration image
        return self.__image
    
    def display_hist(self, array, chan):
        # affiche un histogramme d'un canal de couleur pour un array RGB
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.hist(array.flat, color=chan, bins=256)
    
    def fitting(self, strips, dose_values):
        # créé une courbe fittée d'étalonnage à partir des valeurs données
        
        # converting to optical density values
        dor = -np.log10(strips[:, 0]) # red curve
        dog = -np.log10(strips[:, 1]) # green curve
        dob = -np.log10(strips[:, 2]) # blue curve
        rsb = dor / dob # black curve
        
        legend_list = [(rsb, 'k', 'ratio red/blue ODs'), (dor, 'r', 'red optical density'), (dog, 'g', 'green optical density'), (dob, 'b', 'blue optical density')]
        
        fig, ax = plt.subplots(figsize=(13, 15))
        for e in legend_list: # fitting the curves
            # calculates natural cubic spline polynomials
            x = e[0]

            cs = interpolate.CubicSpline(x[::-1], dose_values[::-1])

            xb = np.linspace(min(x), max(x), num=100)

            #ax.plot(x, dose_values, e[1]+'*')
            ax.plot(xb, cs(xb), e[1], label=e[2])
        
        ax.set_xlim(left=0)
        xliml, xlimr = ax.get_xlim()
        ax.set_xticks(np.arange(xliml, xlimr, 0.1))
        ax.legend()
        ax.set_ylabel('Absorbed Dose (cGy)')
        ax.set_xlabel('Color (16 bits / channel)')
        
    
    def program(self, dose_values):
        # exécute un programme qui montre des informations pertinentes sur la calibration
        
        self.display_hist(self.get_image().get_array()[:,:,0].flatten(), 'red')
        self.display_hist(self.get_image().get_array()[:,:,1].flatten(), 'green')
        self.display_hist(self.get_image().get_array()[:,:,2].flatten(), 'blue') # doesn't display idk why
        
        self.get_image().show('r')
        
        # strip management
        strips = [0 for i in range(8)] # list of length 8
        thirdx = self.get_image().get_width()//3 # third of the length of a strip

        # the zone of interest on the strip is as wide as a third of the image and 20 pixels high to be sure that none of the transitions betweens different doses are included
        # the width can be reduced if the dose is localized on a smaller area on the filter

        # splits the image in 8 strips
        for i in range(7, -1, -1):
            centy = (2*i+1)*(self.get_image().get_height()//8)//2 # mid-height of the current strip

            zoi = self.get_image().get_array()[centy-10 : centy+10, thirdx : 2*thirdx] # zone of interest
            strips[i] = (np.mean(zoi[:,:,0]), np.mean(zoi[:,:,1]), np.mean(zoi[:,:,2]))

        strips = np.array(strips) # contains the median color of each strip
        
        self.fitting(strips, dose_values)

In [ ]:
# TESTING #
files = ['scan1.tif', 'scan2.tif', 'scan3.tif', 'scan4.tif', 'scan5.tif']
doses = [799.7, 599.8, 399.9, 299.9, 199.9, 100.0, 50.4, 24.8]
test = Calibration(Image3d(files, 'tif1/').get_2dImage())
test.program(doses)